In [1]:
from keras.layers.normalization import BatchNormalization
from keras.models import Sequential
from keras.layers import Dense, LeakyReLU, Dropout, Activation
from keras.optimizers import Adam
from keras import regularizers

Using TensorFlow backend.


In [2]:
import numpy as np # linear algebra
import pandas as pd # csv I/O

IMG_SIZE = 299
MODEL_NAME = 'xception'

DATA_DIR= r'D:\LICENTA\processed_data\size_{size1}x{size2}'.format(size1=IMG_SIZE, size2=IMG_SIZE)
FEATURES_PATH = DATA_DIR +  r'\train_biz_{name}_features.csv'.format(name=MODEL_NAME)

In [3]:
train_photos = pd.read_csv('train_photo_to_biz_ids.csv')
train_photo_to_biz = pd.read_csv('train_photo_to_biz_ids.csv', index_col='photo_id')

train_df = pd.read_csv(FEATURES_PATH)

X = train_df['feature_vector'].values
Y = train_df['label'].values

def convert_label_to_array(str_label):
    str_label = str_label[1:-1]
    str_label = str_label.split(',')
    return [int(x) for x in str_label if len(x) > 0]

def convert_feature_to_vector(str_feature):
    str_feature = str_feature[1:-1]
    str_feature = str_feature.split(',')
    return [float(x) for x in str_feature]

Y = np.array([convert_label_to_array(y) for y in train_df['label']])
X = np.array([convert_feature_to_vector(x) for x in train_df['feature_vector']])

In [4]:
print("X_train: ", X.shape)
print("y_train: ", Y.shape)
print("train_df:")
train_df[0:5]

X_train:  (1996, 2048)
y_train:  (1996,)
train_df:


,business,label,feature_vector
0,1000,"(1, 2, 3, 4, 5, 6, 7)","[0.12353174, 0.056759115, 0.10699466, 0.045236..."
1,1001,"(0, 1, 6, 8)","[0.19606702, 0.030776627, 0.032012787, 0.21252..."
2,100,"(1, 2, 4, 5, 6, 7)","[0.13964602, 0.059296891, 0.10956762, 0.041540..."
3,1006,"(1, 2, 4, 5, 6)","[0.16115737, 0.023083402, 0.10019045, 0.051235..."
4,1010,"(0, 6, 8)","[0.12933584, 0.086438626, 0.062900275, 0.05014..."


In [5]:
from sklearn import preprocessing
from sklearn.preprocessing import MultiLabelBinarizer

mlb = MultiLabelBinarizer()
Y = mlb.fit_transform(Y)

In [6]:
model = Sequential()

model.add(Dense(2500, input_shape=(X.shape[1],)))
model.add(BatchNormalization())
model.add(LeakyReLU())
model.add(Dropout(0.5))

model.add(Dense(1024))
model.add(LeakyReLU())
model.add(Dropout(0.5))

model.add(Dense(365))
model.add(Activation('relu'))
model.add(Dropout(0.5))

model.add(Dense(9))
model.add(Activation('sigmoid'))

In [7]:
from keras import backend as K

def f1(y_true, y_pred):
    def recall(y_true, y_pred):
        """Recall metric.

        Only computes a batch-wise average of recall.

        Computes the recall, a metric for multi-label classification of
        how many relevant items are selected.
        """
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
        recall = true_positives / (possible_positives + K.epsilon())
        return recall

    def precision(y_true, y_pred):
        """Precision metric.

        Only computes a batch-wise average of precision.

        Computes the precision, a metric for multi-label classification of
        how many selected items are relevant.
        """
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
        precision = true_positives / (predicted_positives + K.epsilon())
        return precision
    precision = precision(y_true, y_pred)
    recall = recall(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall))

In [8]:
model.compile(Adam(lr=0.001), loss='binary_crossentropy', metrics=['accuracy', f1])

In [9]:
X = preprocessing.scale(X)

model.fit(X, Y, epochs=50, batch_size=1996, verbose=2)

Epoch 1/50
 - 3s - loss: 1.0883 - acc: 0.5051 - f1: 0.5108
Epoch 2/50
 - 0s - loss: 0.6477 - acc: 0.7388 - f1: 0.7292
Epoch 3/50
 - 0s - loss: 0.5483 - acc: 0.7834 - f1: 0.7728
Epoch 4/50
 - 0s - loss: 0.5070 - acc: 0.7977 - f1: 0.7872
Epoch 5/50
 - 0s - loss: 0.4767 - acc: 0.8040 - f1: 0.7953
Epoch 6/50
 - 0s - loss: 0.4454 - acc: 0.8104 - f1: 0.8017
Epoch 7/50
 - 0s - loss: 0.4154 - acc: 0.8172 - f1: 0.8082
Epoch 8/50
 - 0s - loss: 0.4156 - acc: 0.8137 - f1: 0.8063
Epoch 9/50
 - 0s - loss: 0.4026 - acc: 0.8190 - f1: 0.8115
Epoch 10/50
 - 0s - loss: 0.3934 - acc: 0.8289 - f1: 0.8245
Epoch 11/50
 - 0s - loss: 0.3917 - acc: 0.8270 - f1: 0.8221
Epoch 12/50
 - 0s - loss: 0.3835 - acc: 0.8291 - f1: 0.8219
Epoch 13/50
 - 0s - loss: 0.3780 - acc: 0.8329 - f1: 0.8254
Epoch 14/50
 - 0s - loss: 0.3757 - acc: 0.8341 - f1: 0.8256
Epoch 15/50
 - 0s - loss: 0.3633 - acc: 0.8392 - f1: 0.8305
Epoch 16/50
 - 0s - loss: 0.3623 - acc: 0.8427 - f1: 0.8360
Epoch 17/50
 - 0s - loss: 0.3540 - acc: 0.8446 - 

In [24]:
# Saving the current model
model.save(r'D:\LICENTA\models\svm_{size1}x{size2}\NeuralNet_{name}.h5'.format(size1=IMG_SIZE, size2=IMG_SIZE, name=MODEL_NAME))

In [10]:
TEST_FEATURES_PATH = DATA_DIR +  r'\test_biz_{name}_features.csv'.format(name=MODEL_NAME)

test_photo_to_biz = pd.read_csv('test_photo_to_biz.csv', index_col='photo_id')
test_df = pd.read_csv(TEST_FEATURES_PATH)

X_test = test_df['feature_vector'].values

X_test = np.array([convert_feature_to_vector(x) for x in test_df['feature_vector']])

print('X_test ', X_test.shape)
test_df[0:5]

c:\users\eduar\appdata\local\programs\python\python36\lib\site-packages\numpy\lib\arraysetops.py:463: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


X_test  (10000, 2048)


,business,feature_vector
0,003sg,"[0.11907065, 0.073270835, 0.10785412, 0.073216..."
1,00er5,"[0.13801534, 0.069412991, 0.10356507, 0.093447..."
2,00kad,"[0.11502581, 0.073984087, 0.12335605, 0.070464..."
3,00mc6,"[0.11302663, 0.069561109, 0.10974503, 0.097833..."
4,00q7x,"[0.080705076, 0.051065292, 0.079456739, 0.1045..."


In [23]:
%%time

submission = pd.DataFrame(columns=['business_id','labels'])

X_test = preprocessing.scale(X_test)

y_predict = model.predict(X_test)
y_predict = np.where(y_predict > 0.27, 1, 0)

y_predict_label = mlb.inverse_transform(y_predict)

for i in range(len(test_df)):
    biz = test_df.loc[i]['business']
    label = y_predict_label[i]
    label = str(label)[1:-1].replace(',', ' ')
    submission.loc[i] = [str(biz), label]

with open('submission_NN.csv', 'w') as f:
    submission.to_csv(f, index=False)

Wall time: 52.7 s
